In [ ]:
#!pip install k3d
import time
import k3d
import numpy as np
from numpy import sin,cos,pi
from scipy.io import FortranFile
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import math
import os
from os.path import isfile, join
import gc
from scipy.interpolate import RegularGridInterpolator

nx = 288
ny = 288
nz_tot = 385

xmin, xmax = 0, 1
ymin, ymax = 0, 1
zmin, zmax = 0, 2.5

#pathIn = '/home/idies/workspace/Storage/ggsaltar/Python_Vis_Code/PCon_1_120000.out'
pathIn = './Old/Pcon_1_55000.out'
#files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#files.sort(key = lambda x: int(x[-10:-4]))

#pathIn2 = '/home/idies/workspace/Storage/ggsaltar/Python_Vis_Code/PCon_15_120000.out'
pathIn2 = './Old/Pcon_16_55000.out'
#files2 = [f for f in os.listdir(pathIn2) if isfile(join(pathIn2, f))]
#files2.sort(key = lambda x: int(x[-10:-4]))

In [ ]:
count1=1
count2 = 0

xdim = xmax/nx
ydim = ymax/ny
zdim = zmax/nz_tot

vol = xdim*ydim*zdim

f20 = FortranFile(pathIn, 'r')
farray20 = f20.read_reals().reshape((nx,ny,nz_tot), order='F')
#farray = np.where(farray20<=0,1,farray20)
at = np.zeros((nz_tot,ny,nx))
for nn in range(ny):
    at[:,nn,:] = (np.flip(np.transpose(farray20[:,nn,:]), axis =0))
    
dt = FortranFile(pathIn2, 'r')
dot = dt.read_reals()

dotnum = vol*dot

dot = dot.reshape((nx,ny,nz_tot), order='F')
#farray = np.where(farray20<=0,1,farray20)
dots = np.zeros((nz_tot,ny,nx))
for mm in range(ny):
    dots[:,mm,:] = (np.flip(np.transpose(dot[:,mm,:]), axis =0))
    
dotnumb = np.where(dotnum<=0,0,(dotnum))
dotnumb = dotnumb.astype(int)

dotnumber = sum(dotnumb)

print(dotnumber)

In [ ]:
pts = np.random.uniform(low=0,high=1,size=(nx*ny*nz_tot,4))
pts[:,0] = nz_tot*pts[:,0]
pts[:,1] = ny*pts[:,1]
pts[:,2] = nx*pts[:,2]
pts[:,3] = 0.5*np.max(dots)*pts[:,3]

In [ ]:
interp = np.zeros((nx*ny*nz_tot,3))
interp[:,0]=pts[:,0]
interp[:,1]=pts[:,1]
interp[:,2]=pts[:,2]

x = np.linspace(0,nx,nx,dtype=np.int64)
y = np.linspace(0,ny,ny,dtype=np.int64)
z = np.linspace(0,nz_tot,nz_tot,dtype=np.int64)

interF = RegularGridInterpolator((z,y,x),dots)

test = interF(interp)

pts[:,3] = np.where(pts[:,3]>test[:],-1234,pts[:,3])

ptss = pts.tolist()

In [ ]:
new =[]
for row in ptss:
    if row[3]>-100:
        new.append(row[: 3])
neww = np.array(new)

In [ ]:
neww[:,0]=zmax*neww[:,0]/nz_tot
neww[:,1]=ymax*neww[:,1]/ny
neww[:,2]=xmax*neww[:,2]/nx

neww=np.flip(neww,axis = 1)

np.save('neww',neww)

In [ ]:
neww.shape

In [ ]:
count = 0
count1 = 0
r = 0.5
step = 600
ibeta, fbeta =  math.pi, math.pi
itheta, ftheta =  3*math.pi/2, 3*math.pi
ix, fx = 0.65, 0.25
iy, fy = 0.39, 0.39
iz, fz = 0.36, 0.36
dtheta = ftheta - itheta
dbeta = fbeta - ibeta
dx = fx - ix
dy = fy - iy
dz = fz - iz
a = 60

In [ ]:
neww = np.load('neww.npy')
plot = k3d.plot()
points = k3d.points(neww, point_size = 0.003, shader = '3dSpecular', color = 000000)
microns_20 = k3d.volume(at, color_range=[0,np.max(farray20)],color_map=np.array(k3d.basic_color_maps.Jet,dtype=np.float32), 
                                bounds=[xmin,xmax,ymin,ymax,zmin,zmax],name='14 micron droplet',alpha_coef=a)
#microns_1000 = k3d.volume(dots, color_range=[0,np.max(dots)], color_map=np.array(k3d.basic_color_maps.Jet,dtype=np.float32), 
                               #bounds=[xmin,xmax,ymin,ymax,zmin,zmax],name='1000 micron droplet',alpha_coef=a)
#color_map=np.array(k3d.basic_color_maps.Jet,dtype=np.float32)
plot += points
#plot += microns_1000
plot += microns_20
plot.display()

# @interact(theta=widgets.FloatSlider(value=itheta + dtheta*count/step,min=math.pi,max=3*math.pi,step=0.01),
#           x=widgets.FloatSlider(value=ix + dx*count/step,min=-0.25,max=1.75,step=0.01),
#           y=widgets.FloatSlider(value=iy + dy*count/step,min=-math.pi,max=math.pi,step=0.01),
#           z=widgets.FloatSlider(value=iz + dz*count/step,min=0,max=math.pi,step=0.01))
# def g(theta,x,y,z):        
#     plot.camera_auto_fit = False
#     if (theta<=4):
#         plot.camera = [(0.25*theta-0.25*itheta+ix)+r*cos(theta),y+r*sin(theta),z,
#                             0.25*theta-0.25*itheta+ix,y,z,
#                            0,0,1] 
#     elif (theta<=4.3):
#         plot.camera = [(0.25*theta-0.25*itheta+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*theta + 0.23333*4,iz+0.1*theta -0.1*4,
#                             0.25*theta-0.25*itheta+ix,iy - 0.23333*theta + 0.23333*4,iz+ 0.1*theta - 0.1*4,
#                             0,0,1]  
#     elif (theta<=5.37):
#         plot.camera = [(0.25*theta-0.25*itheta+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*0.3,iz+0.1*0.3 + 0.11*theta - 0.11*4.3,
#                             0.25*theta-0.25*itheta+ix,iy-0.23333*0.3,iz+ 0.1*0.3 + 0.11*theta - 0.11*4.3,
#                             0,0,1] 
#     elif (theta<=6.28):
#         plot.camera = [(0.25*theta-0.25*itheta+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*0.3+0.07692*theta - 0.07692*5.37,iz+0.1*0.3+0.11*1.07,
#                             0.25*theta-0.25*itheta+ix,iy-0.23333*0.3+0.07692*theta-0.07692*5.37,iz+0.1*0.3+0.11*1.07,
#                             0,0,1]           
#     elif (theta<=7.19):
#         plot.camera = [(0.25*(6.28-itheta)+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*0.3+0.91*0.07692 +0.09*theta - 0.09*6.28,iz+0.1*0.3+0.11*1.07,
#                             (0.25*(6.28-itheta)+ix),iy-0.23333*0.3+0.91*0.07692+0.09*theta-0.09*6.28,iz+0.1*0.3+0.11*1.07,
#                             0,0,1]     
#     elif (theta<=8):
#         plot.camera = [(-0.4*theta+0.4*7.19+0.25*(6.28-itheta)+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*0.3+0.15*theta - 0.15*7.19+ 0.09*1.82,iz+0.1*0.3+0.11*1.07-theta*0.0823+0.0823*7.19,
#                             (-0.4*theta+0.4*7.19+0.25*(6.28-itheta)+ix),iy-0.23333*0.3+0.15*theta - 0.15*7.19 + 0.09*1.82,iz+0.1*0.3+0.11*1.07-theta*0.0823+0.0823*7.19,
#                             0,0,1]
#     elif (theta<=8.21):
#         plot.camera = [(-0.3*theta+0.3*8-0.81*0.4+0.25*(6.28-itheta)+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*0.3+0.15*0.81+ 0.09*1.82-0.26334*theta+0.26334*8,iz+0.1*0.3+0.11*1.07-0.81*0.0823-theta*0.2+0.2*8,
#                             (-0.3*theta+0.3*8-0.81*0.4+0.25*(6.28-itheta)+ix),iy-0.23333*0.3+0.15*0.81+0.09*1.82-0.26334*theta+0.26334*8,iz+0.1*0.3+0.11*1.07-0.0823*0.81-theta*0.2+0.2*8,
#                             0,0,1]  
#     elif (theta<=9.42):
#         plot.camera = [(-theta*0.3307+0.3307*8.21-0.3*0.21-0.81*0.4+0.25*(6.28-itheta)+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*0.3+0.15*0.81+ 0.09*1.82-0.26334*0.21-theta*0.13206+0.13206*8.21,iz+0.1*0.3+0.11*1.07-0.0823*0.81-theta*0.033652+8.21*0.033652-0.21*0.2,
#                             (-theta*0.3307+0.3307*8.21-0.3*0.21-0.81*0.4+0.25*(6.28-itheta)+ix),iy-0.23333*0.3+0.15*0.81+0.09*1.82-0.26334*0.21-theta*0.13206+0.13206*8.21,iz+0.1*0.3+0.11*1.07-0.0823*0.81-0.21*0.2-theta*0.033652+8.21*0.033652,
#                             0,0,1]    
#     plot.grid_visible = False

In [ ]:

#print(vol)
#print(np.max(dots)*vol)

In [ ]:
plot.display()

In [ ]:
#plt = k3d.plot()
#iso = k3d.marching_cubes(at,xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax, zmin=zmin, zmax=zmax, 
#                         level=0.0, color = 0xff0000,name = 'Isosurface')
#plt += iso
#@interact(x=widgets.FloatSlider(value = 0, min=0, max = np.max(at),step = 10))
#def g(x):
#    iso.level = x
#plt.axes = ["x (m)","y (m)","z (m)"]
#plt.camera_auto_fit = False
#plt.display()

#@interact(theta=widgets.FloatSlider(value=itheta + dtheta*count/step,min=math.pi,max=3*math.pi,step=0.01),
#         x=widgets.FloatSlider(value=ix + dx*count/step,min=-0.25,max=1.75,step=0.01),
#         y=widgets.FloatSlider(value=iy + dy*count/step,min=-math.pi,max=math.pi,step=0.01),
#         z=widgets.FloatSlider(value=iz + dz*count/step,min=0,max=math.pi,step=0.01))
#def g(theta,x,y,z):        
#    plt.camera_auto_fit = False
#    if (theta<=4):
#        plt.camera = [(0.25*theta-0.25*itheta+ix)+r*cos(theta),y+r*sin(theta),z,
#                            0.25*theta-0.25*itheta+ix,y,z,
#                            0,0,1] 
#    elif (theta<=4.3):
#        plt.camera = [(0.25*theta-0.25*itheta+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*theta + 0.23333*4,iz+0.1*theta -0.1*4,
#                            0.25*theta-0.25*itheta+ix,iy - 0.23333*theta + 0.23333*4,iz+ 0.1*theta - 0.1*4,
#                            0,0,1]  
#    elif (theta<=5.37):
#        plt.camera = [(0.25*theta-0.25*itheta+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*0.3,iz+0.1*0.3 + 0.11*theta - 0.11*4.3,
#                            0.25*theta-0.25*itheta+ix,iy-0.23333*0.3,iz+ 0.1*0.3 + 0.11*theta - 0.11*4.3,
#                            0,0,1] 
#    elif (theta<=6.28):
#        plt.camera = [(0.25*theta-0.25*itheta+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*0.3+0.07692*theta - 0.07692*5.37,iz+0.1*0.3+0.11*1.07,
#                            0.25*theta-0.25*itheta+ix,iy-0.23333*0.3+0.07692*theta-0.07692*5.37,iz+0.1*0.3+0.11*1.07,
#                            0,0,1]           
#    elif (theta<=7.19):
#        plt.camera = [(0.25*(6.28-itheta)+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*0.3+0.91*0.07692 +0.09*theta - 0.09*6.28,iz+0.1*0.3+0.11*1.07,
#                            (0.25*(6.28-itheta)+ix),iy-0.23333*0.3+0.91*0.07692+0.09*theta-0.09*6.28,iz+0.1*0.3+0.11*1.07,
#                            0,0,1]     
#    elif (theta<=8):
#        plt.camera = [(-0.4*theta+0.4*7.19+0.25*(6.28-itheta)+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*0.3+0.15*theta - 0.15*7.19+ 0.09*1.82,iz+0.1*0.3+0.11*1.07-theta*0.0823+0.0823*7.19,
#                            (-0.4*theta+0.4*7.19+0.25*(6.28-itheta)+ix),iy-0.23333*0.3+0.15*theta - 0.15*7.19 + 0.09*1.82,iz+0.1*0.3+0.11*1.07-theta*0.0823+0.0823*7.19,
#                            0,0,1]
#    elif (theta<=8.21):
#        plt.camera = [(-0.3*theta+0.3*8-0.81*0.4+0.25*(6.28-itheta)+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*0.3+0.15*0.81+ 0.09*1.82-0.26334*theta+0.26334*8,iz+0.1*0.3+0.11*1.07-0.81*0.0823-theta*0.2+0.2*8,
#                            (-0.3*theta+0.3*8-0.81*0.4+0.25*(6.28-itheta)+ix),iy-0.23333*0.3+0.15*0.81+0.09*1.82-0.26334*theta+0.26334*8,iz+0.1*0.3+0.11*1.07-0.0823*0.81-theta*0.2+0.2*8,
#                            0,0,1]  
#    elif (theta<=9.42):
#        plt.camera = [(-theta*0.3307+0.3307*8.21-0.3*0.21-0.81*0.4+0.25*(6.28-itheta)+ix)+r*cos(theta),iy+r*sin(theta)- 0.23333*0.3+0.15*0.81+ 0.09*1.82-0.26334*0.21-theta*0.13206+0.13206*8.21,iz+0.1*0.3+0.11*1.07-0.0823*0.81-theta*0.033652+8.21*0.033652-0.21*0.2,
#                            (-theta*0.3307+0.3307*8.21-0.3*0.21-0.81*0.4+0.25*(6.28-itheta)+ix),iy-0.23333*0.3+0.15*0.81+0.09*1.82-0.26334*0.21-theta*0.13206+0.13206*8.21,iz+0.1*0.3+0.11*1.07-0.0823*0.81-0.21*0.2-theta*0.033652+8.21*0.033652,
#                            0,0,1]    
#    plt.grid_visible = False